In [32]:
#%tb
import os
import json
#import argparse
import numpy as np
from utils.dataLoaders import load_glassdoor
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
#from configs import system_configs
from imblearn.over_sampling import RandomOverSampler
import torch.nn as nn
from collections import Counter
from utils.dataLoaders import clean_str
from source.torch_shallow_neural_classifier import TorchShallowNeuralClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import source.utils

In [40]:
#configs of the experiment
ID            = None#= system_configs.ID #name of experiment
dataset    = 'amazon'   #= system_configs.dataset #dataset(s) we are using example: "amazon_microsoft" means using both amazon and microsoft data
data_source  = 'glassdoor' #= system_configs.data_source #data source to run, only glassdoor for now
batch_size   = 256 #= system_configs.batch_size #batch size
learning_rate = 0.001#= system_configs.learning_rate
mode        = 'train'   #= system_configs.mode #train val test
classifier   = 'cnn' #= system_configs.classifier #type of classifier
word_embedding = 'GloVe'#= system_configs.word_embedding#word embedding: GloVe/Bert
rating_mode = 2#= system_configs.rating_mode #binary, tenary or 5 star rating
rebalance  = False#= system_configs.rebalance

GLOVE_HOME = os.path.join('data', 'glove.6B')
glove_lookup = source.utils.glove2dict(os.path.join(GLOVE_HOME, 'glove.6B.50d.txt'))

In [44]:
def get_revirew_embed(reviews, lookup, func=np.sum):
    result = []
    for r in reviews:
        embed = np.array([lookup[w] for w in r if w in lookup]) 
        if len(embed) == 0:
            dim = len(next(iter(lookup.values())))
            e = np.zeros(dim)
        else:
            e = func(embed,  axis = 0)
        result.append(e)
    return np.array(result)
    
def fit_nn_classifier(X, y):   
    basemod = TorchShallowNeuralClassifier()
    cv = 3
    param_grid = {'hidden_dim': [50, 100, 200, 300], 'hidden_activation': [nn.Tanh(), nn.ReLU()]}
    best_mod = source.utils.fit_classifier_with_crossvalidation(X, y, basemod, cv, param_grid)
    return best_mod

In [45]:
#pros cons
X, y = load_glassdoor("all","train", 2, "pros_cons", labelType="pros_cons")
test_X, test_y = load_glassdoor("collections", "train", 2, "pros_cons", labelType = "pros_cons", restrict = 20000)
embed_X = get_revirew_embed(X, glove_lookup, np.mean)
embed_test_X = get_revirew_embed(test_X, glove_lookup, np.mean)

nn_model = fit_nn_classifier(embed_X, y)
test_preds = nn_model.predict(embed_test_X)

Finished epoch 100 of 100; error is 11.697825253009796

Best params: {'hidden_activation': ReLU(), 'hidden_dim': 200}
Best score: 0.651


In [46]:
print("macro-f1:",f1_score(test_y, test_preds, average='macro'))
print(classification_report(test_y, test_preds))

macro-f1: 0.6712825640411333
              precision    recall  f1-score   support

           0       0.66      0.72      0.69     10040
           1       0.69      0.62      0.65      9960

    accuracy                           0.67     20000
   macro avg       0.67      0.67      0.67     20000
weighted avg       0.67      0.67      0.67     20000



In [55]:
#sa
X, y = load_glassdoor("all","train", 2, "pros_cons", labelType="sa")
test_X, test_y = load_glassdoor("collections", "train", 2, "pros_cons", labelType = "sa", restrict = 20000)
embed_X = get_revirew_embed(X, glove_lookup, np.mean)
embed_test_X = get_revirew_embed(test_X, glove_lookup, np.mean)

nn_sa = TorchShallowNeuralClassifier()
nn_sa.fit(embed_X, y)
test_preds_sa = nn_sa.predict(embed_test_X)
print(classification_report(test_y, test_preds_sa))